In [1]:
%load_ext lab_black
%load_ext autotime
import pandas as pd
import numpy as np

time: 242 ms (started: 2022-09-15 22:45:27 -07:00)


A 3D S-curve with a hole data set, used to validate the [PaCMAP method](https://arxiv.org/abs/2012.04456). The function below is taken from the [data_prep function at the PaCMAP github repo](https://github.com/YingfanWang/PaCMAP/blob/d34bfdd644c1dd68e8181c926ff34e98b53b0453/experiments/run_experiments.py):

In [2]:
from sklearn.datasets import make_swiss_roll, make_s_curve


def make_scurvehole():
    X, labels = make_s_curve(n_samples=10000, random_state=20200202)
    anchor = np.array([0, 1, 0])
    indices = np.sum(np.square(X - anchor), axis=1) > 0.3
    X, labels = X[indices], labels[indices]
    return X, labels

time: 380 ms (started: 2022-09-15 22:45:27 -07:00)


In [3]:
data, labels = make_scurvehole()
data, data.shape, labels, labels.shape

(array([[ 0.29630642,  0.69668633,  1.95509293],
        [ 0.42420331,  0.51052323,  1.90556698],
        [ 0.99639638,  0.86050764,  1.08481893],
        ...,
        [ 0.78023631,  0.19581041, -1.62548485],
        [-0.13674143,  1.02601064, -1.99060677],
        [ 0.58515088,  0.67243971, -1.81092444]]),
 (9505, 3),
 array([-3.44241573, -3.57967458, -4.62746802, ...,  2.24654913,
         3.27876385,  2.51652655]),
 (9505,))

time: 11.5 ms (started: 2022-09-15 22:45:27 -07:00)


The `labels` are effectively the coordinate along the longest axis of the curve, so can be used to color each point.

## Data Pipeline

In [4]:
target = pd.DataFrame(dict(label=labels))
target

,label
0,-3.442416
1,-3.579675
2,-4.627468
3,2.432928
4,-0.921451
...,...
9500,-1.864398
9501,0.688607
9502,2.246549
9503,3.278764


time: 10.1 ms (started: 2022-09-15 22:45:27 -07:00)


In [7]:
from drnb.io.pipeline import create_default_pipeline

data_result = create_default_pipeline(check_for_duplicates=True).run(
    "scurvehole",
    data=data,
    target=target,
    tags=["lowdim", "synthetic"],
    url="https://github.com/YingfanWang/PaCMAP",
    verbose=True,
)

[23:01:51] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=58478;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=898024;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#392\392]8;;\

           INFO     Initial data shape: (9505, 3)                                                    ]8;id=954022;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=564923;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#80\80]8;;\

           INFO     Keeping all columns                                                            ]8;id=804133;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=675567;file:///home/james/dev/drnb/src/drnb/preprocess.py#62\62]8;;\

           INFO     Data shape after filtering columns: (9505, 3)                                   ]8;id=239150;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=113727;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#151\151]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=256779;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=926872;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#135\135]8;;\

...

           INFO     Writing csv format to triplets/scurvehole.5.1337.l2.csv                         ]8;id=312117;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=997519;file:///home/james/dev/drnb/src/drnb/io/__init__.py#221\221]8;;\

           INFO     Writing numpy format to triplets/scurvehole.5.1337.idx.npy                      ]8;id=96348;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=877409;file:///home/james/dev/drnb/src/drnb/io/__init__.py#244\244]8;;\

           INFO     Writing numpy format to triplets/scurvehole.5.1337.l2.npy                       ]8;id=908599;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=967287;file:///home/james/dev/drnb/src/drnb/io/__init__.py#244\244]8;;\

           INFO     Writing pipeline result for scurvehole                                          ]8;id=822586;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=759844;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#128\128]8;;\

time: 7.45 s (started: 2022-09-15 23:01:51 -07:00)
